In [1]:
import pandas as pd
import numpy as np
import re
import scipy.stats
import statsmodels.stats.multitest
import matplotlib.pyplot as plt
import seaborn as sns
import CPTAC.Endometrial as CPTAC

Welcome to the CPTAC data service package. This import contains
information about the package. In order to access a specific data set,
import a CPTAC subfolder by either 'import CPTAC.DataName' or 'from
CPTAC import DataName'.
Loading Endometrial CPTAC data:
Loading Dictionary...
Loading Clinical Data...
Loading Acetylation Proteomics Data...
Loading Proteomics Data...
Loading Transcriptomics Data...
Loading CNA Data...
Loading Phosphoproteomics Data...
Loading Somatic Mutation Data...

 ******PLEASE READ******
CPTAC is a community resource project and data are made available
rapidly after generation for community research use. The embargo
allows exploring and utilizing the data, but the data may not be in a
publication until July 1, 2019. Please see
https://proteomics.cancer.gov/data-portal/about/data-use-agreement or
enter embargo() to open the webpage for more details.


In [2]:
gene = 'TP53'
somatic = CPTAC.get_somatic()
proteomics = CPTAC.get_proteomics()
transcriptomics = CPTAC.get_transcriptomics()

hotspot_mutations = ['p.P250L', 'p.R248W', 'p.S241C', 'p.241del','p.K164E', 'p.R249S', 'p.R273H', 'p.R282W',
                    'p.S127Y', 'p.G244D', 'p.A159P']

mutated_hotspot = somatic.loc[(somatic['Location'].isin(hotspot_mutations)) & (somatic['Gene'] == gene)]

# Get a list of patient identifiers for those who have mutations in the hotspot
hotspot_patients_both_keys = mutated_hotspot[['Clinical_Patient_Key', 'Patient_Id']]
hotspot_patients = hotspot_patients_both_keys['Clinical_Patient_Key']

In [3]:
interProtList = ['AIMP2', 'ARIH2', 'ARRB1', 'ASH2L', 'ATG7', 'AXIN1', 'BANP', 'BCL2', 'BCL2L1', 'BCR', 'BHLHE40', 'BRCA2',
           'BRD7', 'BTBD2', 'BTRC', 'CABLES1', 'CARM1', 'CCDC106', 'CDKN1A', 'CEBPB', 'CHD8', 'CREBBP', 'CSE1L', 'CSNK2A1', 'CUL7',
           'CUL9', 'CXXC1', 'DAXX', 'DDX17', 'DDX5', 'DROSHA', 'DUSP26', 'DVL2', 'E6', 'EP300', 'ETS2', 'FBXO11',
           'FOXO3', 'GSK3B', 'GTF2H1', 'HDAC1', 'HIPK1', 'HIPK2', 'HMGB1', 'HNRNPK', 'HRMT1L2', 'HSP82', 'HSPA1L', 'HSPA9', 'HSPB1',
           'HTT', 'HUWE1', 'IFI16', 'IFI205B', 'ING4', 'IKBKB', 'IP6K2', 'JMJD6', 'KAT5', 'KAT8', 'KDM1A', 'KMT2E', 'LAMA4',
           'MAGEA2B', 'MAGEC2', 'MAP1B', 'MAPK11', 'MAPKAPK5', 'MDM2', 'MDM4', 'MKRN1', 'MPDZ', 'MT1A', 'NCL', 'NCOR2',
           'NFYA', 'NFYB', 'NOC2L', 'NOL3', 'NPM1', 'NR0B2', 'NR4A1', 'NRDC', 'NSP1', 'NUAK1', 'NUMB', 'OTUB1', 
           'PARD3', 'PARP1', 'PBK', 'PHB', 'PIAS1', 'PIAS2', 'PIAS4', 'PIN1', 'PLK1', 'PML', 'PPIF', 'PPP1CC',
           'PPP1R13L', 'PPP2R1A', 'PPP2R5C', 'PRKCD', 'PSME3', 'PTK2', 'RAD51', 'RBPJ', 'RCHY1', 'RFWD3', 'RING1',
           'RPS3', 'RYBP', 'S100A1', 'S100A2', 'S100A4', 'S100B', 'SAFB', 'SETD7', 'SFN', 'SIN3A', 'SIRT1', 'SMAD2',
           'SMYD2', 'SNAT1', 'SOX4', 'SP1', 'SREBF2', 'SRPK1', 'SUMO1', 'SYVN1', 'TAF1', 'TBP', 'TCF4', 'TOE1', 'TP53BP1',
           'TP53BP2', 'TP53INP1', 'TP63', 'TPT1', 'TRIM24', 'TWIST1', 'UBC', 'UBE3A', 'UHRF2', 'USP42', 'USP7', 'VDR', 'VRK1',
           'WRN', 'WWOX', 'XPO1', 'XRCC6', 'YWHAG', 'YWHAZ', 'ZNF420', 'ATM', 'BAX', 'BCL2L11', 'CCNA1', 'CCNA2', 'CCNB1', 
           'CCNB2', 'CCND1', 'CCND3', 'CCNE1', 'CCNE2', 'CDC25A', 'CDC6', 'CDK2', 'CDKN1B', 'CDKN2A', 'CHEK1', 
           'CHEK2', 'CITED2', 'CKS1B', 'MCM5', 'MRE11A', 'PCNA', 'RB1', 'ARMC10', 'BANP', 'CDKN2AIP', 'NUAK1', 'STK11',
           'LKB1', 'UHRF2', 'E4F1', 'MAML1', 'MKRN1', 'PML', 'FBXO42', 'ATM', 'PP2A', 'PPP2R5C', 'PPP2R2A', 'AURKA', 'DAXX',
           'BRD7', 'TRIM24', 'L3MBTL1', 'GRK5', 'CAK', 'CDK7', 'MAT1', 'CDK5', 'AURKB', 'SETD2', 'UHRF2', 'NOC2L', 'PTK2',
           'FAK1', 'PTK2B', 'PYK2', 'PRKCG', 'PPIF', 'SNAI1', 'KAT6A', 'ANKRD2', 'RFFL', 'RNF34', 'MTA1', 'COP1', 'CCAR2',
           'MORC3', 'POU4F2', 'NOP53', 'AFG1L', 'UBD', 'TAF6', 'FATS', 'C10ORF90']
duplicate_len = len(interProtList)
interProtList = np.array(interProtList)
interProtList = np.unique(interProtList)
unique_len = len(interProtList)
interProtList = interProtList.tolist()
print(duplicate_len)
print(unique_len)

231
217


# Comparison of Transcriptomic data and Proteomic data

### All Mutations vs. Wildtype

In [4]:
proteomics_all_mut_wt_tstat = {}
transcriptomics_all_mut_wt_tstat = {}
pcutoff = 0.05 / len(interProtList)

for protein in interProtList:
    if (protein != 'idx') and (protein != gene) and protein in proteomics.columns:
        # Create dataframe with mutation status of TP53 with proteomics of the given protein
        p53_mutations_protein = CPTAC.compare_mutations(proteomics, protein, gene)
        # Create a column showing the location of TP53 mutation
        p53_mutations_protein['Within_Hotspot'] = False
        p53_mutations_protein.loc[hotspot_patients, 'Within_Hotspot'] = True
        p53_mutations_protein.loc[p53_mutations_protein['Mutation'] == 'Wildtype', 'Within_Hotspot'] = 'Wildtype'
        p53_mutations_protein = p53_mutations_protein.loc[p53_mutations_protein['Sample_Status'] == 'Tumor'].dropna(axis=0)
        # Perform the ttest
        mutated_tp53 = p53_mutations_protein.loc[p53_mutations_protein['Within_Hotspot'] != 'Wildtype']
        wt_tp53 = p53_mutations_protein.loc[p53_mutations_protein['Within_Hotspot'] == 'Wildtype']
        if len(mutated_tp53) < 2: continue
        ttest = scipy.stats.ttest_ind(mutated_tp53[protein], wt_tp53[protein])
        if ttest[1] <= pcutoff:
            proteomics_all_mut_wt_tstat[protein] = ttest[0]
            
for protein in interProtList:
    if (protein != 'idx') and (protein != gene) and protein in transcriptomics.columns:
        # Create dataframe with mutation status of TP53 with proteomics of the given protein
        p53_mutations_protein = CPTAC.compare_mutations(transcriptomics, protein, gene)
        # Create a column showing the location of TP53 mutation
        p53_mutations_protein['Within_Hotspot'] = False
        p53_mutations_protein.loc[hotspot_patients, 'Within_Hotspot'] = True
        p53_mutations_protein.loc[p53_mutations_protein['Mutation'] == 'Wildtype', 'Within_Hotspot'] = 'Wildtype'
        p53_mutations_protein = p53_mutations_protein.loc[p53_mutations_protein['Sample_Status'] == 'Tumor'].dropna(axis=0)
        # Perform the ttest
        mutated_tp53 = p53_mutations_protein.loc[p53_mutations_protein['Within_Hotspot'] != 'Wildtype']
        wt_tp53 = p53_mutations_protein.loc[p53_mutations_protein['Within_Hotspot'] == 'Wildtype']
        if len(mutated_tp53) < 2: continue
        ttest = scipy.stats.ttest_ind(mutated_tp53[protein], wt_tp53[protein])
        if ttest[1] <= pcutoff:
            transcriptomics_all_mut_wt_tstat[protein] = ttest[0]
        
proteomics_shared_tstat = {}
transcriptomics_shared_tstat = {}

for protein in proteomics_all_mut_wt_tstat:
    if protein in transcriptomics_all_mut_wt_tstat:
        proteomics_shared_tstat[protein] = proteomics_all_mut_wt_tstat[protein]
        transcriptomics_shared_tstat[protein] = transcriptomics_all_mut_wt_tstat[protein]
    
print(transcriptomics_shared_tstat)
print(proteomics_shared_tstat)

{'AURKA': 5.866110979625483, 'AURKB': 3.8424162811566838, 'CHD8': 4.44161596807808, 'CSE1L': 5.402486879286059, 'DAXX': 5.551628896552229, 'PLK1': 4.40463916067155, 'XPO1': 4.3694509994659985}
{'AURKA': 4.516151943810517, 'AURKB': 4.025603454556902, 'CHD8': 4.365633574908367, 'CSE1L': 5.401747167638314, 'DAXX': 4.197609155092577, 'PLK1': 4.207271113809839, 'XPO1': 5.667695104485602}


### Non-Hotspot vs. Wildtype

In [5]:
proteomics_nonhs_wt_tstat = {}
transcriptomics_nonhs_wt_tstat = {}
pcutoff = 0.05 / len(interProtList)

for protein in interProtList:
    if (protein != 'idx') and (protein != gene) and protein in proteomics.columns:
        p53_mutations_protein = CPTAC.compare_mutations(proteomics, protein, gene)
        p53_mutations_protein['Within_Hotspot'] = False
        p53_mutations_protein.loc[hotspot_patients, 'Within_Hotspot'] = True
        p53_mutations_protein.loc[p53_mutations_protein['Mutation'] == 'Wildtype', 'Within_Hotspot'] = 'Wildtype'
        p53_mutations_protein = p53_mutations_protein.loc[p53_mutations_protein['Sample_Status'] == 'Tumor'].dropna(axis=0)
        mutated = p53_mutations_protein.loc[p53_mutations_protein['Within_Hotspot'] == False]
        wt = p53_mutations_protein.loc[p53_mutations_protein['Within_Hotspot'] == 'Wildtype']
        if len(mutated) < 2: continue
        ttest = scipy.stats.ttest_ind(mutated[protein], wt[protein])
        if ttest[1] <= pcutoff:
            proteomics_nonhs_wt_tstat[protein] = ttest[0]
            
for protein in interProtList:
    if (protein != 'idx') and (protein != gene) and protein in transcriptomics.columns:
        p53_mutations_protein = CPTAC.compare_mutations(transcriptomics, protein, gene)
        p53_mutations_protein['Within_Hotspot'] = False
        p53_mutations_protein.loc[hotspot_patients, 'Within_Hotspot'] = True
        p53_mutations_protein.loc[p53_mutations_protein['Mutation'] == 'Wildtype', 'Within_Hotspot'] = 'Wildtype'
        p53_mutations_protein = p53_mutations_protein.loc[p53_mutations_protein['Sample_Status'] == 'Tumor'].dropna(axis=0)
        mutated = p53_mutations_protein.loc[p53_mutations_protein['Within_Hotspot'] == False]
        wt = p53_mutations_protein.loc[p53_mutations_protein['Within_Hotspot'] == 'Wildtype']
        if len(mutated) < 2: continue
        ttest = scipy.stats.ttest_ind(mutated[protein], wt[protein])
        if ttest[1] <= pcutoff:
            transcriptomics_nonhs_wt_tstat[protein] = ttest[0]
        
proteomics_shared_tstat = {}
transcriptomics_shared_tstat = {}

for protein in proteomics_nonhs_wt_tstat:
    if protein in transcriptomics_nonhs_wt_tstat:
        proteomics_shared_tstat[protein] = proteomics_nonhs_wt_tstat[protein]
        transcriptomics_shared_tstat[protein] = transcriptomics_nonhs_wt_tstat[protein]

print(transcriptomics_shared_tstat)
print(proteomics_shared_tstat)

{'CHD8': 5.158630450453913}
{'CHD8': 4.4435806856212015}


### Hotspot vs. Wildtype

In [7]:
proteomics_htspt_wt_tstat = {}
transcriptomics_htspt_wt_tstat = {}
pcutoff = 0.05 / len(interProtList)

for protein in interProtList:
    if (protein != 'idx') and (protein != gene) and protein in proteomics.columns:
        # Create dataframe with mutation status of TP53 with proteomics of the given protein
        p53_mutations_protein = CPTAC.compare_mutations(proteomics, protein, gene)
        # Create a column showing the location of TP53 mutation
        p53_mutations_protein['Within_Hotspot'] = False
        p53_mutations_protein.loc[hotspot_patients, 'Within_Hotspot'] = True
        p53_mutations_protein.loc[p53_mutations_protein['Mutation'] == 'Wildtype', 'Within_Hotspot'] = 'Wildtype'
        p53_mutations_protein = p53_mutations_protein.loc[p53_mutations_protein['Sample_Status'] == 'Tumor'].dropna(axis=0)
        # Perform the ttest
        hotspot_tp53 = p53_mutations_protein.loc[p53_mutations_protein['Within_Hotspot'] == True]
        other_mut_tp53 = p53_mutations_protein.loc[p53_mutations_protein['Within_Hotspot'] == 'Wildtype']
        if len(hotspot_tp53) < 2 or len(other_mut_tp53) < 2: continue
        ttest = scipy.stats.ttest_ind(hotspot_tp53[protein], other_mut_tp53[protein])
        # Print significant values
        if ttest[1] <= pcutoff:
            proteomics_htspt_wt_tstat[protein] = ttest[0]
            
for protein in interProtList:
    if (protein != 'idx') and (protein != gene) and protein in transcriptomics.columns:
        # Create dataframe with mutation status of TP53 with proteomics of the given protein
        p53_mutations_protein = CPTAC.compare_mutations(transcriptomics, protein, gene)
        # Create a column showing the location of TP53 mutation
        p53_mutations_protein['Within_Hotspot'] = False
        p53_mutations_protein.loc[hotspot_patients, 'Within_Hotspot'] = True
        p53_mutations_protein.loc[p53_mutations_protein['Mutation'] == 'Wildtype', 'Within_Hotspot'] = 'Wildtype'
        p53_mutations_protein = p53_mutations_protein.loc[p53_mutations_protein['Sample_Status'] == 'Tumor'].dropna(axis=0)
        # Perform the ttest
        hotspot_tp53 = p53_mutations_protein.loc[p53_mutations_protein['Within_Hotspot'] == True]
        other_mut_tp53 = p53_mutations_protein.loc[p53_mutations_protein['Within_Hotspot'] == 'Wildtype']
        if len(hotspot_tp53) < 2 or len(other_mut_tp53) < 2: continue
        ttest = scipy.stats.ttest_ind(hotspot_tp53[protein], other_mut_tp53[protein])
        # Print significant values
        if ttest[1] <= pcutoff:
            transcriptomics_htspt_wt_tstat[protein] = ttest[0]
        
proteomics_shared_tstat = {}
transcriptomics_shared_tstat = {}

for protein in proteomics_htspt_wt_tstat:
    if protein in transcriptomics_htspt_wt_tstat:
        proteomics_shared_tstat[protein] = proteomics_htspt_wt_tstat[protein]
        transcriptomics_shared_tstat[protein] = transcriptomics_htspt_wt_tstat[protein]

print(transcriptomics_shared_tstat)
print(proteomics_shared_tstat)

{'AURKA': 5.04423509098076, 'CSE1L': 4.626510048981035, 'XPO1': 4.2469470217267675}
{'AURKA': 5.726277340642757, 'CSE1L': 4.200504464536369, 'XPO1': 4.567858513862131}
